In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import time
import glob
from libfacedetection.tools import detect_image as di

In [ ]:
# Replace with your image folder path
sourceCode = "C:/Users/Sandip/Desktop/YuNet/libfacedetection/tools/detect_image.py"
configuration = "C:/Users/Sandip/Desktop/YuNet/libfacedetection/configs/yunet_n.py"
weight = "C:/Users/Sandip/Desktop/YuNet/libfacedetection/weights/yunet_n.pth"
image_folder = "C:/Users/Sandip/Desktop/YuNet/Data"
outIm = "C:/Users/Sandip/Desktop/YuNet/OutDir/"
outFile = "C:/Users/Sandip/Desktop/YuNet/OutDir/XLFile"
imrs = "C:/Users/Sandip/Desktop/YuNet/OutDir/imrs"

# Resize/downscale factor for detection (e.g., 10.0 -> 1/10 size for detection)
fc = 10.0

# Ensure output folders exist
os.makedirs(outIm, exist_ok=True)
os.makedirs(outFile, exist_ok=True)
os.makedirs(imrs, exist_ok=True)


In [ ]:
def file_formatting(filename):
    splt = filename.split('_')
    # Defaults
    sub_ID = splt[0] if len(splt) > 0 else ""
    im_type = splt[1].lower() if len(splt) > 1 else "out"
    device_code = splt[2].lower() if len(splt) > 2 else "m"
    distance = splt[3] if len(splt) > 3 else ""
    # original 5th token as raw category (may be 'spoof1')
    raw_cat = splt[4].lower() if len(splt) > 4 else ""

    # Map indoor/outdoor
    if im_type == "in":
        im_T = "indoor"
    else:
        im_T = "outdoor"

    # Map device: 'c' -> camera, else Mobile
    im_De = "camera" if device_code == "c" else "Mobile"

    # Determine category normalized + spoof_type + prop
    # We scan the rest for exact 'prop', 'live', or tokens starting with 'spoof'
    rest = [t.lower() for t in splt[5:]] if len(splt) > 5 else []
    prop = "prop" if any(t == "prop" for t in rest) else ""
    spoof_token = ""
    # order: prop > live > spoof*
    if prop == "prop":
        category = "prop"
    elif any(t == "live" for t in ([raw_cat] + rest)):
        category = "live"
    else:
        # capture exact spoof token from raw_cat or rest
        candidates = [t for t in ([raw_cat] + rest) if t.startswith("spoof")]
        if candidates:
            category = "spoof"
            spoof_token = candidates[0]
        else:
            category = raw_cat  # fallback to original

    return sub_ID, im_T, im_De, distance, category, spoof_token, prop


In [ ]:
# Gather files
files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg','.jpeg','.png','.bmp','.tif','.tiff','.webp'))]
files = sorted(files)

# Prepare results table similar to original (with added 'spoof_type' and 'prop')
results = pd.DataFrame(columns=[
    "subject number", "image name", "indoor/outdoor", "device",
    "distance", "category", "spoof_type", "prop",
    "detection", "face box coordinate[xmin,ymin,xmax,ymax]",
    "detection time (s)", "Original image size", "Face detection size (w*h)"
])

j = 0  # row index

for file in files:
    im_path = os.path.join(image_folder, file)

    # Read original
    img = cv2.imread(im_path)
    if img is None:
        # Could not read image; keep a row for traceability
        sub_ID, im_T, im_De, distance, category, spoof_type, prop = file_formatting(file)
        results.loc[j, "subject number"] = sub_ID
        results.loc[j, "image name"] = file
        results.loc[j, "indoor/outdoor"] = im_T
        results.loc[j, "device"] = im_De
        results.loc[j, "distance"] = distance
        results.loc[j, "category"] = category
        results.loc[j, "spoof_type"] = spoof_type
        results.loc[j, "prop"] = prop
        results.loc[j, "detection"] = "Read Error"
        results.loc[j, "face box coordinate[xmin,ymin,xmax,ymax]"] = ""
        results.loc[j, "detection time (s)"] = ""
        results.loc[j, "Original image size"] = ""
        results.loc[j, "Face detection size (w*h)"] = ""
        j += 1
        continue

    height, width = img.shape[:2]
    orig_size_str = f"{width} X {height}"

    # Resize for detection by factor fc and save resized copy
    new_w = max(1, int(round(width / fc)))
    new_h = max(1, int(round(height / fc)))
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    resized_path = os.path.join(imrs, file)
    cv2.imwrite(resized_path, resized)

    # Detect on resized copy using user's di signature
    t0 = time.time()
    try:
        bboxes = di.get_bbox(configuration, weight, resized_path)
    except TypeError:
        # Fallback to image-in memory signature if needed
        bboxes = di.get_bbox(resized)
    except Exception as e:
        bboxes = None
        detect_error = str(e)
    dt = time.time() - t0

    # Standardize bboxes to Nx4 [x,y,w,h] or Nx4 [x1,y1,x2,y2]; upscale to original
    boxes_xyxy = []  # list of [xmin,ymin,xmax,ymax]

    if bboxes is None or (hasattr(bboxes, "shape") and bboxes.size == 0) or (isinstance(bboxes, (list, tuple)) and len(bboxes) == 0):
        # No detections
        sub_ID, im_T, im_De, distance, category, spoof_type, prop = file_formatting(file)
        results.loc[j, "subject number"] = sub_ID
        results.loc[j, "image name"] = file
        results.loc[j, "indoor/outdoor"] = im_T
        results.loc[j, "device"] = im_De
        results.loc[j, "distance"] = distance
        results.loc[j, "category"] = category
        results.loc[j, "spoof_type"] = spoof_type
        results.loc[j, "prop"] = prop
        results.loc[j, "detection"] = "No Detection"
        results.loc[j, "face box coordinate[xmin,ymin,xmax,ymax]"] = ""
        results.loc[j, "detection time (s)"] = round(dt, 4)
        results.loc[j, "Original image size"] = orig_size_str
        results.loc[j, "Face detection size (w*h)"] = ""
        j += 1
        continue

    import numpy as np
    arr = np.array(bboxes)
    if arr.ndim == 1:
        arr = arr.reshape(1, -1)

    # Interpret columns
    for row in arr:
        if row.shape[0] >= 4:
            x0, y0, a, b = row[:4].tolist()
            # Heuristic: if a and b look like width/height (positive and not surpassing resized dims), treat as xywh
            # Otherwise assume xyxy.
            if a > 0 and b > 0 and a <= new_w and b <= new_h:
                # xywh -> to xyxy
                xmin_r, ymin_r = int(x0), int(y0)
                xmax_r, ymax_r = int(x0 + a), int(y0 + b)
            else:
                # xyxy
                x1, y1, x2, y2 = int(x0), int(y0), int(a), int(b)
                xmin_r, ymin_r = min(x1, x2), min(y1, y2)
                xmax_r, ymax_r = max(x1, x2), max(y1, y2)
            # upscale to original
            xmin = int(round(xmin_r * fc))
            ymin = int(round(ymin_r * fc))
            xmax = int(round(xmax_r * fc))
            ymax = int(round(ymax_r * fc))
            boxes_xyxy.append([xmin, ymin, xmax, ymax])

    # Draw on original and save
    annotated_path = os.path.join(outIm, file)
    vis = img.copy()
    for (xmin, ymin, xmax, ymax) in boxes_xyxy:
        cv2.rectangle(vis, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.imwrite(annotated_path, vis)

    # Write one row per face
    sub_ID, im_T, im_De, distance, category, spoof_type, prop = file_formatting(file)
    for (xmin, ymin, xmax, ymax) in boxes_xyxy:
        w_box = max(0, xmax - xmin)
        h_box = max(0, ymax - ymin)
        results.loc[j, "subject number"] = sub_ID
        results.loc[j, "image name"] = file
        results.loc[j, "indoor/outdoor"] = im_T
        results.loc[j, "device"] = im_De
        results.loc[j, "distance"] = distance
        results.loc[j, "category"] = category
        results.loc[j, "spoof_type"] = spoof_type
        results.loc[j, "prop"] = prop
        results.loc[j, "detection"] = "Detected"
        results.loc[j, "face box coordinate[xmin,ymin,xmax,ymax]"] = f"[{xmin},{ymin},{xmax},{ymax}]"
        results.loc[j, "detection time (s)"] = round(dt, 4)
        results.loc[j, "Original image size"] = orig_size_str
        results.loc[j, "Face detection size (w*h)"] = f"{w_box}*{h_box}"
        j += 1

# Save Excel + CSV with your original base name
#out_csv = os.path.join(outFile, "YuNet_New.csv")
out_xlsx = os.path.join(outFile, "YuNet_New.xlsx")
#results.to_csv(out_csv, index=False)
try:
    results.to_excel(out_xlsx, index=False)
except Exception as e:
    print(f"[WARN] Could not write Excel: {e}")
#print(f"[OK] Wrote: {out_csv}")
print(f"[OK] Wrote: {out_xlsx}")
